# Create Initial 30B Row Taxi Data in Lakehouse (Y,M,D Paritioned)

In [0]:
 # Azure Blob Storage access info
 blob_account_name = "azureopendatastorage"
 blob_container_name = "nyctlc"
 blob_relative_path = "yellow"
    
 # Construct connection path
 wasbs_path = f'wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow'
 print(wasbs_path)
    
 # Read parquet data from Azure Blob Storage path
 blob_df = spark.read.parquet(wasbs_path)


StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 15, Finished, Available, Finished)

wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


## Write First 1B rows to Lakehouse Table

In [0]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month, date_format
 

# Add dataload_datetime column with current timestamp
opt_df = blob_df.withColumn("dataload_datetime", current_timestamp())
    
# Filter columns to exclude any NULL values in storeAndFwdFlag
opt_df = opt_df.filter(opt_df["storeAndFwdFlag"].isNotNull())
    
# Enable V-Orderds
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
# Enable automatic Delta optimized write
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
    
df_output=opt_df.withColumn("day", date_format(col("tpepPickupDateTime"), "dd")).withColumn("year", date_format(col("tpepPickupDateTime"), "yyyy")).withColumn("month", date_format(col("tpepPickupDateTime"), "MM"))
#df_output.select('day','month','year').show()

# Concatenate Date parts into Date key in format YYYYMMDD  (8 CHARACTERS)
from pyspark.sql.functions import concat, col
df_output = df_output.withColumn("DateKey", concat(col('year'), col('month'),col('day')))

#convert string DateKey column 
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
# Create a Spark session
# Convert the 'points' column from string to integer
df_output = df_output.withColumn("DateKey", df_output["DateKey"].cast(IntegerType()))
display(df_output)

# Load the filtered data into a Delta table
table_name = "yellow_taxi_opt_1billionTable5B"  # New table name
df_output.write.mode("overwrite").partitionBy('year','month','day').format('delta').saveAsTable(table_name)


StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2260b591-52e7-4631-bfe9-db000e9a96f1)

In [0]:
# Change the value after From to your lakehousename.tablename
df = spark.sql("SELECT * FROM LakehousePerfLoad1.yellow_taxi_opt_1billionTable5B")
display(df)

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4fffe0cc-07c2-4eeb-b794-9facfacb4812)

Additional Rows -Configurable


In [0]:
# Create 29B more rows in table_name. You can change the range by adjusting the 29 number
for i in range(5):
    df_output.write.mode("append").partitionBy('year','month','day').format('delta').saveAsTable(table_name)  
#when I used 56 it completed in a hour and 17 min. 

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 18, Finished, Available, Finished)

# Following Python to Add Unique ID to Each Row after Full Table Build

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Initialize Spark session
#spark = SparkSession.builder.appName("UniqueIDExample").getOrCreate()

# Assign unique integer to each row
df = df.withColumn("UniqueID", monotonically_increasing_id().cast("integer"))
display(df)

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0830dbc-b095-4aea-81d2-bc0b217141bd)

# Next Cell Adds Random Integer to Data Frame For use as SKU ID

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
import random

# Define UDF to generate random integers
def random_int():
    return random.randint(1, 500000)

random_int_udf = udf(random_int, IntegerType())

# Add new column with random integers
df = df.withColumn("RandomInt", random_int_udf())

# Show the DataFrame
display(df)

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 701669f4-fe5b-4cff-b1f4-2848a502acd8)

If you get an error in the following cell, ignore it and proceed.

In [0]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month, date_format
 
# Enable V-Orderds
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
# Enable automatic Delta optimized write
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
    
# Load the filtered data into a Delta table -- Change the table name to yours if you changed it.
table_name = "yellow_taxi_opt_1billionTable5B"  # New table name
df.write.mode("overwrite").partitionBy('year','month').format('delta').saveAsTable(table_name)


StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 21, Finished, Available, Finished)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
Cell In[62], line 11
      9 # Load the filtered data into a Delta table -- Change the table name to yours if you changed it.
     10 table_name = "yellow_taxi_opt_1billionTable5B"  # New table name
---> 11 df.write.mode("overwrite").partitionBy('year','month').format('delta').saveAsTable(table_name)

File /opt/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py:1586, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
   1584 if format is not None:
   1585     self.format(format)
-> 1586 self._jwrite.saveAsTable(name)

File ~/cluster-env/trident_env/lib/python3.11/site-packages/py4j/java_gateway.py:1322, in JavaMember.__call__(self, *args)
   1316 command = proto.CALL_COMMAND_NAME +\
   1317     self.command_header +\
   1318     args_command +\
   1319     proto.END_COMMAND_PART
   1321 answer = self.ga

In [0]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 23, Finished, Available, Finished)

In [0]:
 
# Enable V-Orderds
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")
    
# Enable automatic Delta optimized write
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 25, Finished, Available, Finished)

Loading Data from Table you wrote to Parquet Files

In [0]:
df = spark.sql("SELECT * FROM LakehousePerfLoad1.yellow_taxi_opt_1billiontable5b")
display(df)
df.write.mode("overwrite").parquet("abfss://WorkspacePerfTestingPoc@onelake.dfs.fabric.microsoft.com/LakehousePerfLoad1.Lakehouse/Files/5Billion")

StatementMeta(, 761da538-27e4-4cb3-8398-b1aa52880607, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4f6b1602-ef03-494b-9f2a-e14e602b10f8)

In [0]:
## Yourlakehouse.table
df = spark.sql("SELECT * FROM Lakehousename.tablename LIMIT 1000")
display(df)

StatementMeta(, c1a560e5-f238-4f0c-b1f5-9c8ef5101373, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dc4d65fe-3aa1-4330-8fef-d8d66322d561)